In [1]:
%load_ext sql

In [28]:
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4
from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LinearRegression
from dateutil.parser import parse

In [44]:
air_temps = xr.open_dataset('data/air_temperature_2021-04-29T12_00_00Z_108000.nc', engine='netcdf4')
subset = air_temps.sel(longitude=slice(12.436523437499998, 14.084472656249998), latitude=slice(52.05924589011585, 52.9751081817353))
df = subset.to_dataframe()
msk = np.random.rand(len(df)) < 0.8
df.pop('latitude_longitude')
df.pop('forecast_period')
df['forecast_reference_time'] = df['forecast_reference_time'].apply(str)
df['forecast_reference_time'] = df['forecast_reference_time'].apply(parse)
df['forecast_reference_time'] = df['forecast_reference_time'].values.astype(float)
df['time'] = df['time'].apply(str)
df['time'] = df['time'].apply(parse)
df['time'] = df['time'].values.astype(float)
train = df[msk]
test = df[~msk]

y_train = train.pop('air_temperature')
y_test = test.pop('air_temperature')
X_train = train
X_test = test


In [45]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [50]:
print('Predicted temps are:')
print(model.predict(X_test.head()))
print('Actual temps are:')
print(list(y_test.head()))

Predicted temps are:
[241.59616136 241.59616136 241.59616136 241.59616136 241.59616136]
Actual temps are:
[276.875, 268.25, 262.0, 218.25, 218.625]


In [51]:
%sql sqlite://

In [52]:
%%sql
CREATE TABLE sales
(
    key       varchar(6),
    ts        timestamp,
    product   integer,
    completed boolean,
    price     float
);
INSERT INTO sales
VALUES ('sale_1', '2019-11-08 00:00', 0, TRUE, 1.1),
       ('sale_2', '2019-11-08 01:00', 0, FALSE, 1.2),
       ('sale_3', '2019-11-08 01:00', 0, TRUE, 1.3),
       ('sale_4', '2019-11-08 01:00', 1, FALSE, 1.4),
       ('sale_5', '2019-11-08 02:00', 1, TRUE, 1.5),
       ('sale_6', '2019-11-08 02:00', 1, TRUE, 1.5);

 * sqlite://
Done.
6 rows affected.


[]

In [54]:
%%sql

select * from sales;

 * sqlite://
Done.


key,ts,product,completed,price
sale_1,2019-11-08 00:00,0,1,1.1
sale_2,2019-11-08 01:00,0,0,1.2
sale_3,2019-11-08 01:00,0,1,1.3
sale_4,2019-11-08 01:00,1,0,1.4
sale_5,2019-11-08 02:00,1,1,1.5
sale_6,2019-11-08 02:00,1,1,1.5
